In [1]:

import math
import re
import numpy as np
import sys
from google.colab import drive

In [2]:

# Mount google drive
drive.mount("/content/drive")

Mounted at /content/drive


#Initialization:
In this we do not have a given model. So we are assuming the A , B and pi

In [3]:
# number of states in the model
N = 2

# number of observation symbols
M = 27

# state transition probabilities
A = np.array([[0.47468, 0.52532], [0.51656, 0.48344]])

# observation probability matrix
B = np.array([[0.03735, 0.03408, 0.03455, 0.03828, 0.03782, 0.03922, 0.03688, 0.03408, 0.03875, 0.04062, 0.03735, 0.03968, 0.03548, 0.03735, 0.04062, 0.03595, 0.03641, 0.03408, 0.04062, 0.03548, 0.03922, 0.04062, 0.03455, 0.03595, 0.03408, 0.03408, 0.03688], 
              [0.03909, 0.03537,  0.03537, 0.03909, 0.03583,  0.03630, 0.04048, 0.03537, 0.03816, 0.03909, 0.03490, 0.03723, 0.03537, 0.03909, 0.03397, 0.03397, 0.03816, 0.03676, 0.04048, 0.03443, 0.03537, 0.03955, 0.03816,  0.03723,  0.03769, 0.03955, 0.03397]])

# initial state distribution
pi = np.array([[0.51316, 0.48684]])

# no. of iterations
max_iters = 100

old_log_prob = -math.inf

#Observation Sequence

In [5]:
O = np.zeros(50000, dtype=int)

file = "/content/drive/My Drive/book-war-and-peace.txt"

f = open(file, "r")
content = f.read()
f.close()

# Remove the punctuations
content = re.sub('[^a-zA-Z]', " ", content)
content = " ".join(content.split()).lower()[:50000]
print(content)

# map the letters and the space to the respective indexes
dictionary = {}
for i in range(26):
    # ASCII value for the each character in the alphabet
    dictionary[chr(i+97)] = i
dictionary[" "] = 26

# adding the values to the observation list/sequence
for i in range(len(content)):
    O[i] = dictionary[content[i]]
print(O)

# length of the observation sequence
T = len(O)
print(T)

chapter i well prince so genoa and lucca are now just family estates of the buonapartes but i warn you if you don t tell me that this means war if you still try to defend the infamies and horrors perpetrated by that antichrist i really believe he is antichrist i will have nothing more to do with you and you are no longer my friend no longer my faithful slave as you call yourself but how do you do i see i have frightened you sit down and tell me all the news it was in july and the speaker was the well known anna pavlovna scherer maid of honor and favorite of the empress marya fedorovna with these words she greeted prince vasili kuragin a man of high rank and importance who was the first to arrive at her reception anna pavlovna had had a cough for some days she was as she said suffering from la grippe grippe being then a new word in st petersburg used only by the elite all her invitations without exception written in french and delivered by a scarlet liveried footman that morning ran as 

#Baum- Welch Algorithm

In [6]:

def baumWelch(A, B, pi, T, M, O, old_log_prob):
    N = 2
    # alpha-pass
    c = np.zeros([T, 1])
    alpha = np.zeros([T, N])
    c[0][0] = 0

    for i in range(N):
        alpha[0][i] = pi[0][i]*B[i][O[0]]
        c[0][0] = c[0][0] + alpha[0][i]

    # scale the alpha[0][i]
    c[0][0] = 1/c[0][0]
    for i in range(N):
        alpha[0][i] = c[0][0]*alpha[0][i]

    # compute alpha[t][i]
    for t in range(1, T):
        c[t][0] = 0
        for i in range(N):
            alpha[t][i] = 0
            for j in range(N):
                alpha[t][i] = alpha[t][i] + alpha[t-1][j]*A[j][i]

            alpha[t][i] = alpha[t][i]*B[i][O[t]]
            c[t][0] = c[t][0] + alpha[t][i]

        c[t][0] = 1/c[t][0]
        for i in range(N):
            alpha[t][i] = c[t][0]*alpha[t][i]


    # beta-pass
    beta = np.zeros([T, N])

    for i in range(N):
        beta[T-1][i] = c[T-1][0]

    for t in range(T-2, -1, -1):
        for i in range(N):
            beta[t][i] = 0
            for j in range(N):
                beta[t][i] = beta[t][i] + A[i][j]*B[j][O[t+1]]*beta[t+1][j]
            
            # scale the beta[t][i]
            beta[t][i] = c[t][0]*beta[t][i]


    # compute gamma[t][i][j] and gamma[t][i]
    di_gamma = np.zeros([T, N, N])
    gamma = np.zeros([T, N])

    for t in range(T-1):
        for i in range(N):
            gamma[t][i] = 0
            for j in range(N):
                di_gamma[t][i][j] = alpha[t][i]*A[i][j]*B[j][O[t+1]]*beta[t+1][j]
                gamma[t][i] = gamma[t][i] + di_gamma[t][i][j]


    # handling the case for gamma[T-1][i]
    for i in range(N):
        gamma[T-1][i] = alpha[T-1][i]


    # re-estimate A, B and pi
    # re-estimate pi
    for i in range(N):
        pi[0][i] = gamma[0][i]

    # re-estimate A
    for i in range(N):
        denominator = 0
        for t in range(T-1):
            denominator = denominator + gamma[t][i]

        for j in range(N):
            numerator = 0
            for t in range(T-1):
                numerator = numerator + di_gamma[t][i][j]
            A[i][j] = numerator/denominator


    # re-estimate B
    for i in range(N):
        denominator = 0
        for t in range(T):
            denominator = denominator + gamma[t][i]
        
        for j in range(M):
            numerator = 0
            for t in range(T):
                if O[t] == j:
                    numerator = numerator + gamma[t][i]
            
            B[i][j] = numerator/denominator


    # compute log(P(O/lambda))
    log_prob = 0
    for i in range(T):
        log_prob = log_prob + math.log(c[i][0])

    log_prob = -1 * log_prob

    if log_prob > old_log_prob:
        old_log_prob = log_prob

    return A, B, pi, old_log_prob
     

In [7]:
a, b, p, log_probability = baumWelch(A, B, pi, T, M, O, old_log_prob)

for i in range(100):
    sys.stdout.write("\r")
    sys.stdout.write("Percentage completed: %d%%" % (((i+1) * 100)/ max_iters))
    sys.stdout.flush()
    a, b, p, log_probability = baumWelch(a, b, p, T, M, O, log_probability)

print("\n")
print("\nState transition matrix: \n", a)
print("\n Observation probability matrix: \n", b)
print("\n Pi: \n", p)
print("\nLog Probability log(P(O/lambda)): ", log_probability)
     

Percentage completed: 100%


State transition matrix: 
 [[0.28421231 0.71578769]
 [0.71072112 0.28927888]]

 Observation probability matrix: 
 [[1.38994659e-01 2.09881065e-15 1.13484046e-04 7.60455293e-10
  2.05852734e-01 1.51160699e-18 1.00715886e-03 1.23743118e-04
  1.20107545e-01 1.93312710e-24 6.04729808e-07 5.49282431e-03
  2.62729730e-18 1.76158719e-06 1.21151256e-01 1.31814074e-03
  3.67590784e-19 2.04274598e-11 9.38351765e-08 9.71223151e-05
  3.79118779e-02 1.01558265e-21 5.33569824e-22 1.10587405e-08
  2.96478079e-05 4.35168232e-33 3.67797335e-01]
 [2.25210347e-03 2.29184173e-02 4.05027423e-02 7.10670220e-02
  3.98424104e-08 2.98138716e-02 3.09263379e-02 9.87252651e-02
  2.11361922e-09 1.71389903e-03 1.20764090e-02 6.15874955e-02
  3.89812385e-02 1.19373312e-01 5.63168677e-09 3.55599686e-02
  1.39503410e-03 9.37064332e-02 1.03312146e-01 1.36656766e-01
  4.52696236e-03 2.28785592e-02 3.51150011e-02 2.51105040e-03
  3.17773399e-02 1.03631104e-03 1.58626650e-03]]

 Pi: 
 [[1.9595